# Lab No 3: Geovisualization Techniques - Data Viz - Part 1: 4 Challenges

## Challenge 1

In [ ]:
import pandas as pd

# Reading the csv file
listings =pd.read_csv("/Users/hoitik/Desktop/Assginment_1/Lab 3/listings.csv")

# Create a subset to extract useful attribute
subset_listings = listings[['id', 
                            'neighbourhood_cleansed',
                            'latitude',
                            'longitude',
                            'property_type',
                            'room_type',
                            'bedrooms',
                            'price',
                            'number_of_reviews']]

# Delete any \$, in the price column, allows us to use .astype to make it numeric 
subset_listings.loc[subset_listings.index, 'price'] = subset_listings['price'].replace('[\$,]', '', regex=True).astype(float)

# Drop any NaN value
subset_listings = subset_listings.dropna()

In [ ]:
# Create a function 'data_description' to produce the mean 
# and standard deviation for the numeric data frame


def data_description(x): 
    # Select only the data with a numeric data types
    numOnly_df = subset_listings.select_dtypes(include=['number'])

    # Define stats as the outcome of the fuction
    # which included the .mean and .count fuction
    stats = pd.DataFrame({
        "Mean": numOnly_df.mean(),
        "Counts": numOnly_df.count()
    })

    
    return stats

# plug in the subset and run the fuction
result = data_description(subset_listings.price)

# Display result
print(result)

## Challenge 2

In [ ]:
import requests
import pandas as pd
import geopandas as gpd

# access the Qualified Property Sales record in New York
# The API has a default limit of 1000 rows of data, 
# adding in ?$limit=1400 in the URL allows us to access all 1400 rows 
url_price = "https://data.cityofnewyork.us/resource/8wi4-bsy4.json?$limit=1400"   
response_price = requests.get(url_price)

In [ ]:
# Convering the response to json and creates data frame 
data_price = response_price.json()
price_pd = pd.DataFrame(data_price)

In [ ]:
# Checking for NaN vaule
nan_in_column_Long = price_pd['longitude'].isna().any()
nan_in_column_Lat = price_pd['latitude'].isna().any()

#Display result
print(nan_in_column_Long,nan_in_column_Lat)

In [ ]:
# remove NaN value
clean_price_pd = price_pd.dropna()

In [ ]:
# Clear unwanted attribute
df_P = clean_price_pd[['latitude', 'longitude', 'price','cap_rate','council_district']]

# Ensure price and cap_rate to be numeric
df_P['price'] = df_P['price'].replace('[\$,]', '', regex=True).astype(float)
df_P.loc[:, 'cap_rate'] = pd.to_numeric(df_P['cap_rate'], errors='coerce')

In [ ]:
import folium
from folium.plugins import HeatMap

# By using folium map
# create a heatMap for the distribution of houses sales

# Creating the Map and Making New York the central viewpoint
m = folium.Map(location=[40.7128, -74.0060], zoom_start=10) 


heat_data = list(zip(df_P['latitude'], df_P['longitude']))
HeatMap(heat_data, min_opacity=0.4, radius=10, blur=15).add_to(m)

# Display map
m

In [ ]:
# To better illustrate house price
# I found the shape file of NYC council district

# Reading the Shape file
districts_gdf = gpd.read_file('/Users/hoitik/Desktop/Assginment_1/Lab 3/nycc.shp')

In [ ]:
# Create a geo data frame for the NYC house price data
gdf_p = gpd.GeoDataFrame(df_P, 
                          geometry=gpd.points_from_xy(df_P['longitude'], df_P['latitude']),
                          crs="EPSG:4326")  

# Making sure the two geo data frames use the same CRS
gdf_p = gdf_p.to_crs(districts_gdf.crs)

# Display the first five row 
gdf_p.head()

In [ ]:
# Use sjoin to combine the two geo data frames
gdf_joined = gpd.sjoin(gdf_p, districts_gdf, how="inner", predicate="within")

# Create a new column with the average house price grouped by council district
avg_price_per_district = gdf_joined.groupby('CounDist')['price'].mean().reset_index()

# Merge the new column into the NYC council district map
districts_gdf = districts_gdf.merge(avg_price_per_district, how='left', on='CounDist')

# Display the first five row 
districts_gdf.head()

In [ ]:
# Creating the Base Map and Making New York the central viewpoint
m_P_1 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# Adding in the data
folium.Choropleth(
    geo_data=districts_gdf, # Using the merged map  
    data=districts_gdf,
    columns=['CounDist', 'price'],  
    key_on='feature.properties.CounDist',  
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average House Price in NYC by District',
    bins=7,
    reset=True
).add_to(m_P_1)

# Displaying the map
m_P_1

In [ ]:
# Runing the mean, median and standard deviation for price and capitalization rate
price_mean = df_P['price'].mean()
price_median = df_P['price'].median()
price_std = df_P['price'].std()

cap_rate_mean = df_P['cap_rate'].mean()
cap_rate_median = df_P['cap_rate'].median()
cap_rate_std = df_P['cap_rate'].std()


print(f"Price - Mean: {price_mean:.2f}, Median: {price_median:.2f}, Std Dev: {price_std:.2f}")
print(f"Cap Rate - Mean: {cap_rate_mean:.2f}, Median: {cap_rate_median:.2f}, Std Dev: {cap_rate_std:.2f}")


The descriptive statistics result shows a high deviation of house prices in New York. 
The standard deviation for the house price and capitalization rate in New York is extremely high. Even our result from the Map of New York house prices by district shows that the house prices in Upper West Manhattan in comparison much higher than other districts in New York. It means that outliers might exist in the data, pulling the variance away from the mean. To gain a better understanding of the New York City housing situation, outliers should be removed and help normalize the data.

In [ ]:
import numpy as np

# To remove outliers from the capitalization rate, 
# we have to find out the first quantile and third quantile and 
# then keep the data between the two

Q1_cap_rate = df_P['cap_rate'].quantile(0.25) 
Q3_cap_rate = df_P['cap_rate'].quantile(0.75)
IQR_cap_rate = Q3_cap_rate - Q1_cap_rate

# Define the outlier bounds by the Tukey Fences method
lower_bound_cap_rate = Q1_cap_rate - 1.5 * IQR_cap_rate
upper_bound_cap_rate = Q3_cap_rate + 1.5 * IQR_cap_rate

# Now we only keep the capitalization rate falls between the 'lower_bound_cap_rate' and 'upper_bound_cap_rate'
df_P_filtered = df_P[(df_P['cap_rate'] >= lower_bound_cap_rate) & (df_P['cap_rate'] <= upper_bound_cap_rate)]



In [ ]:
# Now we do the same with the price

Q1_price = df_P['price'].quantile(0.25)
Q3_price = df_P['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price

lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price


df_P_filtered = df_P[(df_P['cap_rate'] >= lower_bound_cap_rate) & (df_P['cap_rate'] <= upper_bound_cap_rate) &
                     (df_P['price'] >= lower_bound_price) & (df_P['price'] <= upper_bound_price)]


In [ ]:
# Ensuring both cap_rate and price are numeric
df_P_filtered['cap_rate'] = pd.to_numeric(df_P_filtered['cap_rate'], errors='coerce')
df_P_filtered['price'] = pd.to_numeric(df_P_filtered['price'], errors='coerce')


In [ ]:
# Now we run the descriptive statistic again
price_mean = df_P_filtered['price'].mean()
price_median = df_P_filtered['price'].median()
price_std = df_P_filtered['price'].std()

cap_rate_mean = df_P_filtered['cap_rate'].mean()
cap_rate_median = df_P_filtered['cap_rate'].median()
cap_rate_std = df_P_filtered['cap_rate'].std()


print(f"Price - Mean: {price_mean:.2f}, Median: {price_median:.2f}, Std Dev: {price_std:.2f}")
print(f"Cap Rate - Mean: {cap_rate_mean:.2f}, Median: {cap_rate_median:.2f}, Std Dev: {cap_rate_std:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Creating two subplots in a single figure
fig, axes = plt.subplots(1, 2, figsize=(15, 6))  

# Plotting a histogram for the normalized capitalization rate
sns.histplot(df_P_filtered['cap_rate'], bins=50, kde=True, color='blue', ax=axes[0])
axes[0].set_title('Distribution of Houses Capitalization Rate (Outliers Excluded)')
axes[0].set_xlabel('Capitalization Rate')
axes[0].set_ylabel('Frequency')

# Plotting a histogram for the normalized New York Houses price
sns.histplot(df_P_filtered['price'], bins=50, kde=True, color='green', ax=axes[1])
axes[1].set_title('Distribution of House Prices (Outliers Excluded)')
axes[1].set_xlabel('Price(USD10m)')
axes[1].set_ylabel('Frequency')

# Display figure
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# With the normalized capitalization rate and normalized New York Houses price,
# standard deviation for both variances shrank almost 10 times,  
# We can now test for the correlation between the price and capitalization rate

# Poltting a the scatter plot with regression line
plt.figure(figsize=(10, 6))
sns.regplot(data=df_P_filtered, x='cap_rate', y='price', color='red', scatter_kws={'alpha': 0.5}, line_kws={'color': 'blue'})


plt.title('Correlation between House Price and Capitalization Rate (Outliers Excluded)')
plt.xlabel('Capitalization Rate')
plt.ylabel('Price(USD10m)')

# Display figure
plt.show()


With the regression line in the graph, it is clear that house prices and Capitalization Rates are negatively correlated. The more expensive the houses, the lower the Capitalization rate. It shows that cheaper houses have higher profitability in investing, in comparison to the current US bank rate of 4.25%, the New York house capitalization Rate holds a mean of 5% and nearly 8% for houses priced at USD 5-6m. Prices for cheaper houses might increase as the demand rises for investment, especially for houses in Upper Manhattan and Washington Heights (where house sales are mostly distributed). Housing in New York City could be less affordable.

## Challenge 3

In [ ]:
import requests
import pandas as pd
import geopandas as gpd

# access the API with a URL path, 
# Since the website has defaulted a limit of 1000 rows of data per entry
# adding ?$limit= to the URL allows us to set the limit on our own
# Now the limit is set as 2m, which is much closer to the actual dataset size

url_collisions = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=2000000"
response = requests.get(url_collisions)
data = response.json()

# creating a data frame for data
collisions_pd = pd.DataFrame(data)

# To reduce the processing time, only keep the attribute needed
# I have chosen the no. of the person killed, 
# and the contributing factor of the collision for further descriptive statistics measurement 
df = collisions_pd[['latitude', 'longitude', 'number_of_persons_killed', 'number_of_cyclist_killed','contributing_factor_vehicle_1']]

In [ ]:
# converting the number_of_persons_killed and number_of_cyclist_killed into nnumeric
df['number_of_persons_killed'] = pd.to_numeric(df['number_of_persons_killed'], errors='coerce')
df['number_of_cyclist_killed'] = pd.to_numeric(df['number_of_cyclist_killed'], errors='coerce')

# drop any row with NaN value in longitude/latitude
df = df.dropna(subset=['latitude', 'longitude'])

In [ ]:
import folium
import pandas as pd

# Now we create a map to show the location of which collisions are recorded with death
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Create a loop to test the value of number_of_persons_killed and number_of_cyclist_killed
for _, row in df.iterrows():
    persons_killed = row['number_of_persons_killed']
    cyclists_killed = row['number_of_cyclist_killed']

    if cyclists_killed > 0:  # if number_of_cyclists_killed is bigger than 0, a blue CircleMarker will be shown
        color = 'blue'
    elif persons_killed > 0: # if number_of_persons_killed is bigger than 0, a red CircleMarker will be shown
        color = 'red'
    else:
        continue  # Nothing will happen if the row fails to meet the condition 

# Plotting the Map with folium map
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1,
        color=color,
        fill=True,
        fill_color=color, # the fill_color will be determined by the loop function
        
    ).add_to(m)

# Display the map
m

In [ ]:
# calculate the descriptive statistics for the no. of persons and cyclist killed
mean_cyclist_killed = df_drop['number_of_cyclist_killed'].mean()
std_cyclist_killed = df_drop['number_of_cyclist_killed'].std()

mean_persons_killedd = df_drop['number_of_persons_killed'].mean()
std_persons_killed = df_drop['number_of_persons_killed'].std()

print(f"persons_killed - Mean: {mean_persons_killedd}, Std Dev: {std_persons_killed}")
print(f"cyclist_killed - Mean: {mean_cyclist_killed}, Std Dev: {std_cyclist_killed}")

In [ ]:
# To better understand the correlation between different contributing factors and no. of person killed
# First create a subset to drop all rows with 'NaN' value or 'Unspecified' for contributing_factor_vehicle_1
df_drop = df.dropna(subset=['contributing_factor_vehicle_1'])
df_drop = df_drop[df_drop['contributing_factor_vehicle_1'] != 'Unspecified']

# review the first five rows of the data
df_drop.head()

In [ ]:
# Use .value_counts to count the frequency of each contributing factors
contributing_factor  = df_drop['contributing_factor_vehicle_1'].value_counts()

# View the frequency of each contributing factor
contributing_factor

In [ ]:
import matplotlib.pyplot as plt


# Since there are too many contributing factors, 
# It is complicated to test them all, 
# result would also be unreliable due to the small sample size of 
# those low frequency contributing factor

# So, I set up a threshold
# If the frequency of the contributing factor is less than 0.5% of the 
# total frequency, it will be placed in 'other' 
threshold = 0.001 * contributing_factor.sum()


major_factors = contributing_factor[contributing_factor >= threshold]
other_factors = contributing_factor[contributing_factor < threshold].sum()
final_factors = pd.concat([major_factors, pd.Series({'Other': other_factors})])# placing other_factors into 'other'

# plotting the bar chart
plt.figure(figsize=(12, 6))
final_factors.sort_values().plot(kind='barh', color='navy', edgecolor='black')


plt.xlabel("Number of Incidents")
plt.ylabel("Contributing Factor")
plt.title("Distribution of Contributing Factors to collision (Grouped)")
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Show the chart
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# By using groupby function, find out the average number of persons_killed
# for each contributing factor
correlation = df_drop.groupby('contributing_factor_vehicle_1')['number_of_persons_killed'].mean()

# Only show the factors who passed the threshold
correlation = correlation[correlation.index.isin(final_factors.index)]

# Sort the data in an ascending way
correlation = correlation.sort_values(ascending=False)

# Plotting the graph
plt.figure(figsize=(12, 6))
correlation.plot(kind='bar', color='navy', alpha=1)

plt.title("Average Number of Persons Killed by Each Contributing Factor")
plt.xlabel("Contributing Factor")
plt.ylabel("Average Number of Persons Killed")
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.6)

#Display the graph
plt.show()


## Challenge 4

1. Both LongBoard and Datasher libraries are powerful for visualizing large datasets in a short amount of time. Yet, LongBoard is much flexible compared to Datasher. LongBoard allows you to create all kinds of layers, such as Solid Polygon Layer and Scatter Plot Layer. It is friendly to both vector and raster data, and the outcome of LongBoard trends is to be an interactive or customizable map. whereas, datashader works better with raster data and provides high effectiveness in processing massive data sets with million of points. It works great in handling the loss of information due to overplotting issues in data visualization. Nonetheless, I found LongBoard with more exciting functionality of its flexibility.

2. The original file of the dataset is a 12.8GB file with 33m rows and over 50 columns, it is a dataset of the US national-wide congestion record from 2016 to 2022. To reduce the size of the file, I created a subset with 6 columns 'Start_Lat','Start_Lng','Severity','DelayFromTypicalTraffic(mins)','DelayFromFreeFlowSpeed(mins)' and 'Congestion_Speed'] and exported the file as 'us_congestion_2016_2022_dropped', around 1.7GB. here is a onedrive link for you to access : 
https://universityofstandrews907-my.sharepoint.com/:f:/r/personal/chtu1_st-andrews_ac_uk/Documents/Data?csf=1&web=1&e=H0ujR5

3. potential problem : Urban area and its outstrike trends to be Traffic Congestion hotspots

In [ ]:
#I also included the code that I used to create the new file

#import pandas as pd

#file_path = '/Users/hoitik/Desktop/UA/us_congestion_2016_2022.csv'
#df_CH4 = pd.read_csv(file_path)
#df_CH4.head()

#df_filtered = df_CH4[['Start_Lat', 'Start_Lng', 'Severity','DelayFromTypicalTraffic(mins)','DelayFromFreeFlowSpeed(mins)','Congestion_Speed']]
#df_filtered.head()

#output_path = '/Users/hoitik/Desktop/UA/us_congestion_2016_2022_dropped.csv'
#df_filtered.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
file_path = '/Users/hoitik/Desktop/UA/us_congestion_2016_2022_dropped.csv'
df_CH4 = pd.read_csv(file_path)

In [ ]:
# Review the size of the dataset
len(df_CH4)

In [ ]:
# Import 
import holoviews as hv, pandas as pd, colorcet as cc
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

In [ ]:
import datashader.transfer_functions as tf
from datashader.utils import lnglat_to_meters
import cartopy.crs as ccrs

# The EsriImagery was not working at the first place, 
# Then I found that the projection for the EsriImagery is mercator
# Then I Converted the unit to meter for longitude and latitude,
# so that It could fit the mercator projection
df_CH4['x'], df_CH4['y'] = lnglat_to_meters(df_CH4['Start_Lng'], df_CH4['Start_Lat'])

#ploting the map
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
points = hv.Points(df_CH4, ['x', 'y'])
Con = datashade(points, cmap=cc.fire, width=900, height=480)
map_tiles * Con

In [ ]:
import hvplot.pandas

map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
plot = df_CH4.hvplot(
    'x', 'y',
    kind='scatter',
    rasterize=True,
    cmap=cc.fire,
    cnorm='eq_hist'
)
map_tiles * plot

In [ ]:
import colorcet as cc

hv.extension('bokeh')
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')


colors = {'Slow': 'red', 'Moderate': 'yellow', 'Fast': 'green'}
alpha_value = 0.1

layers = []

for speed, color in colors.items():
    df_subset = df_CH4[df_CH4['Congestion_Speed'] == speed]
    points = hv.Points(df_subset, ['x', 'y'])
    shaded = datashade(points, cmap=[color], width=900, height=480)
    layers.append(shaded)


Con =  layers[0] *layers[1] * layers[2]


map_tiles * Con

# Congestion speed record the speed during the congestion and classify into three group
# each group is given a colour
# If you zoom in on the map, most urban areas are dominant by yellow (Moderate) and red (Slow)
# Besides, the map also shows that the mid-west part of the US is also dominant by red
# showing the severity of congestion in these area